_______________
# 01. Introduction to NumPy & Vectorization
_______________
Now, since we got familiar with **Python**, it's about the time to make some proper use of it **for scientific computing**. In the following lab we will learn how to improve the speed of computations using [`NumPy`](http://www.numpy.org/) - a Python package for numeric computation that offers optimized vectorized data routines.

* __Read the text__ and have a go with the cells and the objects created by them
* __Attempt Exercises at the bottom of the notebook__
_______________
**`Note`** Basic knowledge of `python` is assumed for this part of the course, so make sure you went through and understood the material that was presented earlier. Here are some additional links in case you want to hear it from someone else once again, or in case you need a follow-up after going through this notebook: 

- [`Introduction to Jupyter notebooks`](http://bebi103.caltech.edu/2015/tutorials/t0b_intro_to_jupyter_notebooks.html)

- [`Introduction to Python for scientific computing`](http://bebi103.caltech.edu/2015/tutorials/t1a_intro_to_python.html)

- [`Python/Numpy tutorial`](http://cs231n.github.io/python-numpy-tutorial/#python)

_______________
## `Data Science Stack`
Apart from `NumPy`, we will also cover many additional libraries used for data science in Python, so you may start getting to know the better:

* [`numpy`](http://www.numpy.org/): scientific computing by using array objects


* [`pandas`](http://pandas.pydata.org/): data structures and data analysis tools


* [`matplotlib`](http://matplotlib.org/): plotting library (similar to MATLAB's plot interface)


* [`scikit-learn`](http://scikit-learn.org/stable/) machine learning library implementing many learning algorithms and useful tools.


* [`seaborn`](https://seaborn.github.io/index.html): data visualisation library which works on top of matplotlib


* [`scipy`](https://www.scipy.org/) a library based on NumPy that extends its functionality - in case you need any functions related to linear algebra, differential calculus, and signal processing.

______________
## `Morning/Evening Reads`
A list of links that are worth to follow and look through when you are having a cup of coffee in the morning or are trying to make yourself fall asleep (just joking, there's quite a lot of interesting content about AI, ML, DS, and etc., both *technical* and not).
- [`Towards Data Science`](https://towardsdatascience.com/) - articles & blog posts on data science (very broadly speaking). Often conceptual and/or ideological, but very often contains nice code starters. 
- [`KDNuggets`](https://www.kdnuggets.com/news/index.html) - an online platform on business analytics, big data, data mining, and data science. I find it more technical than TDS.
- [`Kaggle`](https://www.kaggle.com/) - a platform for machine learning competitions (you may win some prizes here, who knows). If you're really into code/solution digging (not blog post quality documentation though). 

_______________
### `Imports`
As per usual, we start by importing the packages that we will be using later. It's generally a good practice to do so at the top of a file. 

    If you have troubles importing any of these packages, make sure they are properly installed (`README` within the root of this repository).

In [120]:
import os
import sys
import numpy as np
import autotime
%load_ext autotime

ModuleNotFoundError: No module named 'autotime'

# ============ Numpy  ============
[Numpy](http://www.numpy.org) is a powerful scientific computing library that provides numerous methods for working with n-dimensional arrays, which you will find highly useful in data science & machine learning. The following `Numpy` introduction is largely based on this [tutorial](http://cs231n.github.io/python-numpy-tutorial/#numpy), though some of the explanations were also borrowed from the [INF1CG](https://www.learn.ed.ac.uk/webapps/blackboard/content/listContent.jsp?course_id=_72370_1&content_id=_4289282_1) course labs (UoE).

## Why Vectorize?
`Vectorization` refers to the process of rewriting an iterative program (a program that has loops) in such a way that no loops remain. Instead of sequentially performing computations, a vectorized program performs subsets of operations at once (for trivial tasks all operations might be applied at once). 

    Vectorization is a very important and useful concept in data-science and machine learning, as problem formulation in a vectorized form can lead to extremelly large speed improvements.

Before getting into the details of how to perform computations using `NumPy`, let's see *how much faster* a vectorized operation can be. Assume that you have a large number of data stored in ```long_list``` and you want to calculate the sum of all elements in it:

In [ ]:
long_list = list(range(5000000))

We have seen previously that it is possible to simply **loop through** a Python data structure in order to get the sum of the list elements. We only have to store the partial sum while we iterate through the list:

In [ ]:
partial_sum = 0

for number in [1,2,3,4,5]:
    partial_sum = partial_sum + number 
    #this sort of operation is performed all the time so there exists a shorthand
    #if you want you can also write partial_sum += number

partial_sum

And since this sort of operation might be useful in the future we might want to make it a function:

In [ ]:
def my_list_sum(list_of_numbers):
    partial_sum = 0
    for number in list_of_numbers:
        partial_sum += number
    return partial_sum

my_list_sum([1,2,3,4,5])

In [ ]:
my_list_sum(long_list)

This approach is very simple and easy to understand and computing it didn't really take *that long*. Still, if we want to compare implementations we need to calculate how long it took. For that, we can use an IPython [magic command](http://ipython.readthedocs.io/en/stable/interactive/magics.html) 

```
%%timeit
```

``timeit`` calculates the average time it takes to execute a Python expression for a number of runs (and takes care of a number of issues that make estimating processing time tedious). We can use the magic command in a cell like so:

In [ ]:
%%timeit
my_list_sum(long_list)

So as we've said before, it didn't really take long for this list - on a (rather slow) laptop around 31 [milliseconds](https://en.wikipedia.org/wiki/Millisecond).

Summing is a very common operation and of course there exists a build-in operator for summing in Python - we can sum elements of a list using Python's own ``sum()`` implementation.
Let's check how long it takes us to sum the elements using Python's own implementation:

In [ ]:
%%timeit
sum(long_list)

On the same laptop our self-made loop takes roughly 5 times longer (your results might vary but the Python version should be considerably faster).

#### Why are built-ins faster?

When we write loops, the code is executed as Python code, i.e. the Python interpreter has to translate the Python code into bytecode instructions. The native Python operations (like ```sum, len```) are all written as optimized code and thus do not need the same amount of translation and overhead as non-native Python code.

If there exist a build-in function for the command you want to use (and you do not have very good reasons to do otherwise) use the build-in function, as:

- Less typing and thinking means less errors in your code.
- Native implementations should give you better performance.

### `Arrays` in Python
Python lists can contain different *types* of items, but if we know what type all objects in our collection are, it makes sense to explicitly state the type. In such case, the Python interpreter can take advantage of the type declaration, which results in faster computations.

**`Arrays`** are part of the Python Standard Library and provide a collection that is very similar to lists, but specifies the type of contained objects (and thus restricts all contents to be of that type). We can import the array data structure from the array library with:

In [ ]:
from array import array

To create an array data structure need to specify the type that *all* items in the array will have (in this case I pick the type double, *'d'*), and as second argument the collection of elements that are contained in the array:

In [ ]:
pythonArray = array('d', long_list)

Notice that we cannot declare arrays with items of different types!

In [ ]:
array('d', [1,"this does not work"])

Let's see if adding an array instead of a list improves performance:

In [ ]:
%%timeit
sum(pythonArray)

On the same laptop we didn't get any speed-up compared to the native sum operation! Why is that?
The reason is that Python's native ``sum()`` does operate upon lists. The `Python` interpreter has to translate our array into a list to perform the operation and therefore we do not get any speedups (in fact we should be slightly slower).

As you can see at the [array documentation](https://docs.python.org/3/library/array.html), the number of operations defined on the Python array implementation is very restricted - and all operations that *are implemented* (like ```reverse``` or ```count```) are already heavily optimized for `Python` lists.

This was a bit underwhelming, but let's give it one last chance - in the form of `NumPy`'s implementation of arrays and the ***additional support for vectorized functions***.

In [ ]:
numpyArray = np.array(long_list) #we will discuss NumPy array creation in the next section
np.sum(numpyArray) #notice that this is NumPys sum implementation

In [ ]:
%%timeit
np.sum(numpyArray)

As you can see this is *way faster* than our fastest implementation, and this difference in speed will only increase with the size of our data (If you don't believe it, try a larger range for our long_list).

***Why is NumPy so much faster than the Python array?*** The *NumPy ndarray object is of fixed size* and *all elements are the same datatype* as the Python array. In addition to the array data structure, *Numpy operations are performed as optimized code* on the array data structure.

Changing your iterative (loopy) programs to operate on arrays and use vectorized functions in `NumPy` can drastically improve the performance, whilst also in a shorter amount code.

##  `Arrays` in Numpy
A main Numpy object is a ***N-dimensional array*** [`ndarray`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.html), which serves as a container (grid of values) for large arrays of data of the same type. The objects can be used along with a set of provided universal functions [`ufuncs`](https://docs.scipy.org/doc/numpy/reference/ufuncs.html) to perform a variety of scientific computations efficiently.

- The ***number of dimensions*** is the ***rank*** of the array; 
- The ***shape*** of an array is a tuple of integers giving the ***size of the array along each dimension***. 

**`N.B.`** This use of the word `rank` is not the same as the meaning in linear algebra.

Let's start by having a look at how we can create different forms of arrays in NumPy. To start with, we can initialize numpy arrays from nested Python [lists](http://www.tutorialspoint.com/python/python_lists.htm), and access elements using square brackets:

In [ ]:
a = np.array([1, 2, 3])  # Creates a rank 1 array (i.e. vector)
a

In [ ]:
charArray = np.array(['a', 'b', 'c'])
charArray

In [ ]:
floatArray = np.array([1, 2, 3.0])
floatArray

In [ ]:
boolArray = np.array([True, False])
boolArray

For Python arrays we had to declare the type of the contained elements - here we didn't do any of this, but NumPy has set the type by **inferring the optimal data type when you create an array**.

We can access the data type of a NumPy array with:

In [ ]:
boolArray.dtype  # Prints the type of object a (array)

Notice that this tells you the **type of the contained data**!  If you ask for the type of object ```boolArray``` we instead get ```numpy.ndarray```.

In [ ]:
type(boolArray)  # Prints the type of object a (array)

We can also include an optional argument to explicitly specify the data type, like so:

In [ ]:
a = np.array([1,2,3], dtype='int8')

One of the most basic property of an array we might be interested in is its *shape*. The NumPy function ``numpy.shape`` returns the shape of an array as a tuple of integers. Each number in the tuple denotes the length of the corresponding array dimension. Let's see an example:

In [ ]:
a.shape  # Prints the number of elements for each dimension

In [ ]:
a.ndim   # Prints the number of dimensions of object a (array)

In [ ]:
A = np.array([1,2,3,4,5,6,7,8,9,10])
A.shape

In [ ]:
A.ndim

The array A has length 10 in the first dimension (rows) and no other dimensions.

In [ ]:
B = np.array([[1,2,3],[4,5,6]])
B.shape

In [ ]:
B.ndim

The array B has length 2 in the first dimension (rows) and length 3 in the second dimension (columns).

If instead, we want to know the total number of elements in the array we use ```size```:

In [ ]:
A.size

In [ ]:
B.size

<div class="alert alert-info" role="alert">
<h1>Exercises</h1>

<ol>

<li>
Assume you have a large database of user behavior on a video-streaming platform. You want to store different information about the movies in your database in an array. Since your database is very big, the data type in which you store this data will make a big difference in how much space you have to allocate for it on your server. Have a look at the table of different [NumPy types](https://docs.scipy.org/doc/numpy/user/basics.types.html) and think about which data type you would pick for the following data:

<ul style="list-style-type: none;">
   <li><input type="checkbox"> The number of videos watched by each user.</li>
    <li><input type="checkbox"> The average rating of a each movie (ratings from 0 to 10).</li>
    <li><input type="checkbox"> The size of each movie (in MB, in GB?).</li>
    <li><input type="checkbox"> If a user has watched a specific movie.</li>
    <li><input type="checkbox"> The title of the movie that the user has watched.</li>
    <li><input type="checkbox"> The first letter of the title.</li>
</ul>

</li>
<li>
Check what happens if you declare the following NumPy arrays and explain how NumPy handles these arrays (inspecting the resulting array and using the table on [NumPy types](https://docs.scipy.org/doc/numpy/user/basics.types.html).

<ul style="list-style-type: none;">
   <li><input type="checkbox"> numpy.array([-1,2,3],dtype='uint8')</li>
   <li><input type="checkbox"> numpy.array([50,80,250,256])</li>
    <li><input type="checkbox"> numpy.array([50,80,250,256],dtype='float64')</li>
    <li><input type="checkbox"> numpy.array([50,80,250,256],dtype='uint8')</li>
</ul>
</li>
</ol>

</div>

In [ ]:
2**8

In [ ]:
np.array([-1,2,3],dtype='uint8')

In [ ]:
np.array([50,80,250,256]).dtype

In [ ]:
np.array([50,80,250,256],dtype='float64')

In [ ]:
np.array([50,80,250,256],dtype='uint8')

We can create a N-dimensional array by passing a each row as a a sequence-like object (for example a list). If we want to create a 3x3 array:

In [ ]:
b = np.array([[1,2,3],[4,5,6],[7,8,9]])
b

### Array `Creation`
In addition to `numpy.array`, there are several other ways for creating these objects:

1. **Using some pre-set matrix types** (generally, the first argument refers to the shape of the resulting array).

In [ ]:
# Empty
np.zeros((2, 2))

In [ ]:
# Filled with ones
np.ones((1, 2))

In [ ]:
# Filled with N
np.full((2, 2), 7)

In [ ]:
# Identity matrix
np.eye(5)

In [ ]:
# Filled with random floats between 0 and 1
np.random.random((2, 2))

In [ ]:
# Filled with normally distributed random floats defined using mean and std
mu = 2     # mean
sigma = .2 # std
np.random.normal(mu, sigma, (4,1)), np.random.normal(mu, sigma, 10)

2. **From a list**

In [ ]:
some_list = [1, 4, 6, 8]
e = np.array(some_list)
e

In [ ]:
some_list = [[1, 4, 6, 8], [2, 2, 4, 4]]
f = np.array(some_list, dtype=float)
f

3. **Appending an existing array**

In [ ]:
g = np.array([])
for ii in range(10):
    g = np.append(g, ii)
g

Be careful with data types, as numpy will do some inference on your behalf...it may not be what you want/intended.

In [ ]:
np.append(g, 'hello')

In [ ]:
e

In [ ]:
e.dtype

In [ ]:
np.append(e, 2.0)

In [ ]:
np.append(e, 2.0).dtype

<div class="alert alert-info" role="alert">
<h1>Exercises</h1>
<ol>
<li>
<ul style="list-style-type: none;">
   <li><input type="checkbox"> Create a (3,3) array of ones.</li>
    <li><input type="checkbox"> Create a (7,2) array of zeros.</li>
    <li><input type="checkbox"> Why can't you create a (3,2) identity matrix?</li>
    <li><input type="checkbox"> Create an 1D-array of 25 numbers [0..25] (check numpy.arange).</li>
    <li><input type="checkbox"> Create an 1D-array of 25 evenly spaced numbers between 0,100 (check numpy.linspace).</li>
    <li><input type="checkbox"> What is the difference between arange and linspace?</li>
    <li><input type="checkbox"> What does numpy.logspace do?</li>
    <li><input type="checkbox"> Create a (0:5,0:5) meshgrid (numpy.mgrid).</li>
    <li><input type="checkbox"> Create a (5,5) array of all zeros apart from the diagonal, which is [0,1,2,3,4,5].</li>
    <li><input type="checkbox"> Create a (3,3) array of random values, uniformly distributed between [0,1] (numpy.random).</li>
    
</ul>
</li>
</ol>
</div>

In [ ]:
np.ones((3,3))

In [ ]:
np.zeros((7,2))

In [ ]:
# No such ID matrix
np.eye((3))[:2]

In [ ]:
np.arange(26)

In [ ]:
np.linspace(1,25, num = 25)

In [ ]:
#numpy.logspace(start, stop, num=50, endpoint=True, base=10.0, dtype=None, axis=0)[source]
#Return numbers spaced evenly on a log scale.

np.logspace(1, 25)

In [ ]:
#Create a (0:5,0:5) meshgrid (np.mgrid).
np.mgrid[0:5,0:5]

In [ ]:
def create(N=100):
    a = np.zeros((N, N))
    np.fill_diagonal(a, #Your range object)

In [ ]:
#Create a (100,100) array of all zeros apart from the diagonal, which is [0,1,...,100].
a = np.zeros((6, 6))
np.fill_diagonal(a, [0,1,2,3,4,5])
a

### Array `Indexing & Slicing`

In [ ]:
a = np.array([1,2,3,4,5,6,7,8,9,10])
a

In [ ]:
a[0]  # Select array elements by index (starts at 0)

In [ ]:
a[0] = 5  # Change an element of the array
a

In [ ]:
try:
    a[100]  # Will error
except IndexError as e:
    print('{}'.format(e))
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

Slicing is the most common way to index arrays, and works similarly to Python list indexing, but there are also other options, such as integer and boolean array indexing. 

In [ ]:
a[1:3]

In [ ]:
a[1:6:2]

In [ ]:
a[1::2]

Higher dimensional arrays consist of an array of one-dimensional arrays, i.e. providing a single index will return the n-th element in the first dimension (which is an array for non 1D-arrays).

In [3]:
a = np.array([[1,2,3,4], [5,6,7,8], [9,10,11,12]])
a

array([[ 1,  2,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12]])

In [4]:
a[0]

array([1, 2, 3, 4])

In [5]:
a[1][1]

6

In [6]:
a[1,1]

6

<div class="alert alert-warning" role="alert">
<h1>Warning</h1>

Accessing the index directly with `Array[row, column]` is more efficient then the nested access, `Array[row][column]`. In the nested case the intermediate array `Array[row]` is created and only then accessed, whereas `Array[row, column]` does not create this intermediate result.

</div>

In [7]:
a[:,0]

array([1, 5, 9])

In [8]:
a[:][0]

array([1, 2, 3, 4])

In [9]:
a

array([[ 1,  2,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12]])

In [10]:
b = a[:2, 1:3]
b

array([[2, 3],
       [6, 7]])

In [11]:
a

array([[ 1,  2,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12]])

In [12]:
a.shape

(3, 4)

In [13]:
a[:,:,np.newaxis].shape

(3, 4, 1)

In [14]:
a[:,:,np.newaxis]

array([[[ 1],
        [ 2],
        [ 3],
        [ 4]],

       [[ 5],
        [ 6],
        [ 7],
        [ 8]],

       [[ 9],
        [10],
        [11],
        [12]]])

`Warning` **slice of an array is a view into the same data, so modifying it will modify the original array**. E.g.:

`b[0, 0]` is the same piece of data as `a[0, 1]`, but modifying `b` will modify `a`.

<div class="alert alert-warning" role="alert">
<h1>Warning</h1>

Slicing a lists creates a new object, but **slicing an array creates a reference to the original (sub-) array** (in NumPy called a [view](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.view.html)).


This might lead to some confusion, but we can use this to our advantage for modifying arrays efficiently. By selecting a view on our original data and passing it around we can modify the original data by modifying the view (this is beyond this introduction, but if you are curious have a look at the documentation for an [example](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.view.html).
</div>

In [15]:
a

array([[ 1,  2,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12]])

In [16]:
b

array([[2, 3],
       [6, 7]])

In [17]:
b[0, 0]

2

In [18]:
a[0, 1]

2

In [19]:
b[0, 0] = 77
a[0, 1]

77

In [20]:
c = a.copy()
c

array([[ 1, 77,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12]])

In [21]:
c[0, 0] = 100
a[0, 0]

1

In [22]:
a

array([[ 1, 77,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12]])

In [23]:
a[:,[0,2]]

array([[ 1,  3],
       [ 5,  7],
       [ 9, 11]])

In [24]:
x = np.array(
    [[False, False, False,  True],
     [False,  True, False,  True],
     [False,  True, False,  True]])
x

array([[False, False, False,  True],
       [False,  True, False,  True],
       [False,  True, False,  True]])

In [25]:
a.shape, x.shape

((3, 4), (3, 4))

In [26]:
a

array([[ 1, 77,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12]])

In [27]:
a[x] = 0
a

array([[ 1, 77,  3,  0],
       [ 5,  0,  7,  0],
       [ 9,  0, 11,  0]])

<div class="alert alert-info" role="alert">
<h1>Exercises</h1>

<ol>
    
Have a look at the documentation on [Basic Slicing and Indexing](https://docs.scipy.org/doc/numpy/reference/arrays.indexing.html) to find out what the following slices do on an array A:

<ul style="list-style-type: none;">
   <li><input type="checkbox">  A is 1D: A[-3:3:-1]</li>
   <li><input type="checkbox">  A is 1D: A[3:]</li>
   <li><input type="checkbox">  A is 2D: A[1:]</li>
   <li><input type="checkbox">  A is 1D: A[:]</li>
   <li><input type="checkbox">  A is 2D: A[:]</li>
   <li><input type="checkbox">  A is 1D: A[::2]</li>
   <li><input type="checkbox">  A is 2D: A[::2]</li>
   <li><input type="checkbox">  A is 2D: A[::2,::2]</li>
    
</ul>
</ol>
</div>

### Boolean `Indexing`
With boolean indexing we can select a subset of our array based on a logical condition. For example:

In [28]:
np.arange(55)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54])

In [29]:
A = np.arange(55)
A > 2

array([False, False, False,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

As you can see NumPy applies the logical condition (greater than 2) to each element in the array. This works equally for multidimensional arrays:

In [30]:
A = np.array([np.arange(25),np.arange(25)])
A

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
        16, 17, 18, 19, 20, 21, 22, 23, 24],
       [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
        16, 17, 18, 19, 20, 21, 22, 23, 24]])

In [31]:
A > 2

array([[False, False, False,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [False, False, False,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True]])

We can use the array of type boolean to index subsets of our array like so:

In [32]:
A = np.arange(50)
A

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])

In [33]:
A>5

array([False, False, False, False, False, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])

In [34]:
A[A>5]

array([ 6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
       23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39,
       40, 41, 42, 43, 44, 45, 46, 47, 48, 49])

In [35]:
rule1 = A > 5
rule2 = (A % 2 == 0)
# .....

In [36]:
A[rule1 & rule2]

array([ 6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38,
       40, 42, 44, 46, 48])

In [37]:
A[rule1 | rule2]

array([ 0,  2,  4,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
       37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])

## Arrays `Computing`

We have seen how we can create arrays in NumPy and how we can access individual elements, or larger element collections from NumPy arrays. To finish this lab, we will have a quick look at the possibilities that NumPy provides us to perform optimized computations on arrays. 

As the central data structure in NumPy is the array, the computations upon these n-dimensional arrays belong to the field of [Linear Algebra](https://en.wikipedia.org/wiki/Linear_algebra), and NumPy provides implementations for most common operations, e.g. matrix multiplication, decompositions, determinants, etc..

### Scalars

In [38]:
A = np.ones(4)
A

array([1., 1., 1., 1.])

In [39]:
A + 0.5

array([1.5, 1.5, 1.5, 1.5])

In [40]:
B = np.ones([2,2])
B - 0.3

array([[0.7, 0.7],
       [0.7, 0.7]])

Equally, we can also subtract, divide, multiply  or exponentiate scalars:

In [41]:
A - 0.001 #or A - 1e-3 

array([0.999, 0.999, 0.999, 0.999])

In [42]:
A /3

array([0.33333333, 0.33333333, 0.33333333, 0.33333333])

In [43]:
2.5 * A

array([2.5, 2.5, 2.5, 2.5])

In [44]:
C = np.arange(15)
C

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

In [45]:
C * 3

array([ 0,  3,  6,  9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42])

In [46]:
3 ** C

array([      1,       3,       9,      27,      81,     243,     729,
          2187,    6561,   19683,   59049,  177147,  531441, 1594323,
       4782969])

In [47]:
C ** 3

array([   0,    1,    8,   27,   64,  125,  216,  343,  512,  729, 1000,
       1331, 1728, 2197, 2744])

### Array `Math`
Basic mathematical functions (arithmetic operations) operate **elementwise** on arrays, and are available both as operator overloads and as functions in the numpy module:

In [48]:
x = np.array([[1, 2], [3, 4]], dtype=np.float64)
x

array([[1., 2.],
       [3., 4.]])

In [49]:
y = np.array([[5, 6], [7, 8]], dtype=np.float64)
y

array([[5., 6.],
       [7., 8.]])

#### Elementwise sum, equivalent expressions:

In [50]:
x + y

array([[ 6.,  8.],
       [10., 12.]])

In [51]:
np.add(x, y)

array([[ 6.,  8.],
       [10., 12.]])

#### Elementwise difference, equivalent expressions:

In [52]:
x - y

array([[-4., -4.],
       [-4., -4.]])

In [53]:
np.subtract(x, y)

array([[-4., -4.],
       [-4., -4.]])

#### Elementwise product, equivalent expressions:

In [54]:
x * y

array([[ 5., 12.],
       [21., 32.]])

In [55]:
np.multiply(x, y)

array([[ 5., 12.],
       [21., 32.]])

#### Elementwise division, equivalent expressions:

In [56]:
x / y

array([[0.2       , 0.33333333],
       [0.42857143, 0.5       ]])

In [57]:
np.divide(x, y)

array([[0.2       , 0.33333333],
       [0.42857143, 0.5       ]])

#### Elementwise square root

In [58]:
np.sqrt(x)

array([[1.        , 1.41421356],
       [1.73205081, 2.        ]])

In [59]:
x ** (0.5)

array([[1.        , 1.41421356],
       [1.73205081, 2.        ]])

#### Dot product and matrix multiplicaiton

**`N.B.`** `*` is elementwise multiplication, not matrix multiplication. We instead use the `np.dot` function or `.dot` method to compute the inner products of vectors, to multiply a vector by a matrix, and to multiply matrices. `dot` is available both as a function in the numpy module and as an instance method of array objects:

In [60]:
x = np.array([[1, 2], [3, 4]])
y = np.array([[5, 6], [7, 8]])
v = np.array([9, 10])
w = np.array([11, 12])

##### Matrix vector product

In [61]:
x.dot(v)  # using x's method

array([29, 67])

In [62]:
np.dot(x, v)  # using the numpy function

array([29, 67])

##### Matrix matrix product

In [63]:
x

array([[1, 2],
       [3, 4]])

In [64]:
y

array([[5, 6],
       [7, 8]])

In [65]:
x.dot(y)  # using x's method

array([[19, 22],
       [43, 50]])

In [66]:
np.dot(x, y)  # using the numpy function

array([[19, 22],
       [43, 50]])

### `Mathematical Functions`

Numpy provides many useful functions for performing computations on arrays; one of such is `sum`:

In [67]:
x = np.array([[1, 2], [3, 4]])
x

array([[1, 2],
       [3, 4]])

In [68]:
np.sum(x)  # Compute sum of all elements

10

In [69]:
np.sum(x, axis=0)  # Compute sum of each column - sum *over rows* i.e. dimension 0

array([4, 6])

In [70]:
np.sum(x, axis=1)  # Compute sum of each row - sum *over columns* i.e. dimension 1

array([3, 7])

In [71]:
np.sin(x)

array([[ 0.84147098,  0.90929743],
       [ 0.14112001, -0.7568025 ]])

### `Example` Standardization
Recreate the data in such way that it has a 0 mean and 1 (unit) standard deviation.

In [141]:
data = (np.random.rand(1000)*100).round(2)
data

array([95.32, 39.24, 12.68, 50.12, 71.23, 50.03, 39.82, 55.57, 58.3 ,
       70.3 , 38.7 , 41.99, 96.02, 25.59, 71.18, 62.84, 78.98, 23.57,
       81.01, 61.49, 17.89, 97.66, 22.12, 65.49, 42.46, 85.5 , 16.24,
       50.92, 24.87, 73.13, 68.07, 66.01, 80.02, 40.46, 45.02, 84.43,
       80.65, 63.59, 73.62,  4.24, 26.6 , 66.24, 71.21, 28.75, 92.48,
       89.64, 57.29, 51.95, 73.63, 32.03,  1.26, 67.79, 79.68, 66.85,
       13.99, 41.44, 26.98, 40.67, 89.15, 33.14, 32.53, 45.75, 61.12,
       71.13, 75.89, 62.31, 49.05, 74.33, 79.44, 44.17, 96.6 , 91.71,
       97.61, 39.45, 57.81, 11.25, 46.6 , 36.19, 77.32, 85.23,  9.17,
       60.54, 32.01, 32.52, 93.98, 64.41, 16.04, 82.4 , 64.06, 81.44,
        1.01, 33.55, 42.59, 41.6 , 53.53,  3.46, 26.96, 29.13, 57.37,
       33.55, 45.27, 22.58, 34.05, 22.09, 29.29, 10.78,  2.07, 79.69,
       72.31, 48.34, 68.53, 78.93, 79.62, 12.86, 93.93, 62.09,  5.77,
       45.79, 70.02, 67.05, 75.05, 57.7 , 91.56, 49.85, 79.63, 78.19,
       92.34, 60.48,

In [142]:
data.mean(), data.std()

(51.97408, 28.933755752642966)

In [143]:
# Subtract mean from all elements (gets distance from the mean)
data = data-data.mean()
data

array([ 4.334592e+01, -1.273408e+01, -3.929408e+01, -1.854080e+00,
        1.925592e+01, -1.944080e+00, -1.215408e+01,  3.595920e+00,
        6.325920e+00,  1.832592e+01, -1.327408e+01, -9.984080e+00,
        4.404592e+01, -2.638408e+01,  1.920592e+01,  1.086592e+01,
        2.700592e+01, -2.840408e+01,  2.903592e+01,  9.515920e+00,
       -3.408408e+01,  4.568592e+01, -2.985408e+01,  1.351592e+01,
       -9.514080e+00,  3.352592e+01, -3.573408e+01, -1.054080e+00,
       -2.710408e+01,  2.115592e+01,  1.609592e+01,  1.403592e+01,
        2.804592e+01, -1.151408e+01, -6.954080e+00,  3.245592e+01,
        2.867592e+01,  1.161592e+01,  2.164592e+01, -4.773408e+01,
       -2.537408e+01,  1.426592e+01,  1.923592e+01, -2.322408e+01,
        4.050592e+01,  3.766592e+01,  5.315920e+00, -2.408000e-02,
        2.165592e+01, -1.994408e+01, -5.071408e+01,  1.581592e+01,
        2.770592e+01,  1.487592e+01, -3.798408e+01, -1.053408e+01,
       -2.499408e+01, -1.130408e+01,  3.717592e+01, -1.883408e

In [145]:
# Divide by the standard deviation
data = data/data.std()
data

array([ 1.49810900e+00, -4.40111547e-01, -1.35807050e+00, -6.40801704e-02,
        6.65517473e-01, -6.71907241e-02, -4.20065757e-01,  1.24281135e-01,
        2.18634596e-01,  6.33375085e-01, -4.58774869e-01, -3.45066852e-01,
        1.52230220e+00, -9.11878853e-01,  6.63789387e-01,  3.75544748e-01,
        9.33370705e-01, -9.81693502e-01,  1.00353097e+00,  3.28886443e-01,
       -1.17800400e+00,  1.57898340e+00, -1.03180798e+00,  4.67133272e-01,
       -3.28822849e-01,  1.15871304e+00, -1.23503082e+00, -3.64308045e-02,
       -9.36763282e-01,  7.31184717e-01,  5.56302477e-01,  4.85105360e-01,
        9.69314881e-01, -3.97946264e-01, -2.40344878e-01,  1.12173201e+00,
        9.91088756e-01,  4.01466028e-01,  7.48119953e-01, -1.64977131e+00,
       -8.76971528e-01,  4.93054553e-01,  6.64826238e-01, -8.02663857e-01,
        1.39995375e+00,  1.30179851e+00,  1.83727272e-01, -8.32245914e-04,
        7.48465570e-01, -6.89301457e-01, -1.75276519e+00,  5.46625199e-01,
        9.57563900e-01,  

In [146]:
data.mean(), data.std()

(-2.3092638912203255e-17, 1.0)

For the full list of mathematical functions, check the [documentation](http://docs.scipy.org/doc/numpy/reference/routines.math.html).

### Reshaping 
Apart from computing mathematical functions using arrays, we frequently need to **reshape** or otherwise manipulate data in arrays. The simplest example of this type of operation is **transposing** a matrix; to transpose a matrix, simply use the `T` attribute of an array object:

In [147]:
np.arange(4)

array([0, 1, 2, 3])

In [74]:
x = np.arange(4).reshape((2, 2))
x

array([[0, 1],
       [2, 3]])

In [75]:
x.T

array([[0, 2],
       [1, 3]])

In [76]:
np.transpose(x) # Equivalent expression

array([[0, 2],
       [1, 3]])

In [77]:
# Note that taking the transpose of a rank 1 array (a vector) does nothing:
v = np.array([1, 2, 3])
v

array([1, 2, 3])

In [78]:
v.T

array([1, 2, 3])

In [79]:
x.reshape((4, 1))

array([[0],
       [1],
       [2],
       [3]])

In [80]:
x.reshape((4,))

array([0, 1, 2, 3])

In [81]:
y = np.arange(27).reshape((3, 3, 3))
y

array([[[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8]],

       [[ 9, 10, 11],
        [12, 13, 14],
        [15, 16, 17]],

       [[18, 19, 20],
        [21, 22, 23],
        [24, 25, 26]]])

In [82]:
y.shape

(3, 3, 3)

In [83]:
y.reshape((3, -1))

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8],
       [ 9, 10, 11, 12, 13, 14, 15, 16, 17],
       [18, 19, 20, 21, 22, 23, 24, 25, 26]])

In [84]:
y.reshape((3, -1)).shape

(3, 9)

### Broadcasting

Broadcasting is a powerful mechanism that allows numpy to work with arrays of different shapes when performing arithmetic operations. 

In [85]:
x = np.arange(12).reshape((4, 3))
x

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11]])

In [86]:
x+2

array([[ 2,  3,  4],
       [ 5,  6,  7],
       [ 8,  9, 10],
       [11, 12, 13]])

Broadcasting is especially useful when we have a smaller and a larger arrays, and want to use the smaller one multiple times to perform some operation on the larger. For example, suppose that we want to add a constant vector to each row of a matrix. 

In [87]:
v = np.array([1, 0, 1])
v

array([1, 0, 1])

In [88]:
x + v  # Add v to each row of x using broadcasting

array([[ 1,  1,  3],
       [ 4,  4,  6],
       [ 7,  7,  9],
       [10, 10, 12]])

`x + v` works even though `x` has shape `(4, 3)` and `v` has shape `(3,)` due to broadcasting; this line works as if v actually had shape `(4, 3)`, where each row was a copy of `v`, and the sum was performed elementwise.

Broadcasting two arrays together follows these rules:

* If the arrays do not have the same rank, prepend the shape of the lower rank array with 1s until both shapes have the same length.
* The two arrays are said to be compatible in a dimension if they have the same size in the dimension, or if one of the arrays has size 1 in that dimension.
* The arrays can be broadcast together if they are compatible in all dimensions.
* After broadcasting, each array behaves as if it had shape equal to the elementwise maximum of shapes of the two input arrays.
* In any dimension where one array had size 1 and the other array had size greater than 1, the first array behaves as if it were copied along that dimension.

So be careful with shapes...

In [89]:
y = x.T
y

array([[ 0,  3,  6,  9],
       [ 1,  4,  7, 10],
       [ 2,  5,  8, 11]])

In [90]:
try:
    y + v  # Add v to each column of y using broadcasting...?
except ValueError as e:
    print(e)
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

operands could not be broadcast together with shapes (3,4) (3,) 


And especially careful with vectors!

In [91]:
try:
    y + v.T  # Add v to each column of y using broadcasting...?
except ValueError as e:
    print(e)
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

operands could not be broadcast together with shapes (3,4) (3,) 


In [92]:
y + v.reshape((3, 1))  # Add v to each column of y using broadcasting!

array([[ 1,  4,  7, 10],
       [ 1,  4,  7, 10],
       [ 3,  6,  9, 12]])

In [93]:
print('x shape:', x.shape)
print('v shape:', v.shape)
print('y shape:', y.shape)

x shape: (4, 3)
v shape: (3,)
y shape: (3, 4)


### `Moving Averages` NumPy
Here's an example of moving average calculation with NumPy using the [`cumsum`](https://numpy.org/devdocs/reference/generated/numpy.cumsum.html) function. You may want to check the speed difference as compared to the methods that we designed previously using basic Python.

In [184]:
import random
a = list(range(10))
a

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [185]:
np.cumsum(np.array(a))

array([ 0,  1,  3,  6, 10, 15, 21, 28, 36, 45])

In [186]:
# Our previous moving average implementation using basic Python
def moving_average(list_of_integers, N):
    
    result=[]
    for idx, number in enumerate(list_of_integers):
        
        numbers = list_of_integers[:idx+1]
        moving_numbers = numbers[-N:]
        moving_mean = round(sum(moving_numbers)/(len(moving_numbers)), 4)
        result.append(moving_mean)
    
    return result

In [187]:
%%timeit
moving_average(a, 3)

8.69 µs ± 138 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [188]:
# Moving average using numpy and buil-in cumsum method
def moving_average_np(a, n=3) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

In [189]:
%%timeit
moving_average_np(a)

12 µs ± 272 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


This is slower than the original! Quite unexpected, but let's try doing the same with a larger amount of data (more operations to apply) to make sure this is the case all the time.

In [196]:
a = list(range(10000))

In [197]:
%%timeit
moving_average(a, 3)

141 ms ± 1.88 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [198]:
%%timeit
moving_average_np(a, 3)

711 µs ± 8.17 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Well that's much better, `NumPy` version performs at tenfolds faster. And that's not everything, we have not converted our `a` into a `NumPy` array before performing the operation:

In [199]:
a = np.array(a)

In [200]:
%%timeit
moving_average_np(a, 3)

62.4 µs ± 1.34 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


And as a cherry on top, let's assume we have 50000 of aggregated customer order values ranging from 100 to 1000000 dollars cents (1 to 10000 dollars). What is more, let's say these numbers have deviate a lot throughout the time and that your team observed definite weekly trends within it. You decided to smoothen out the stats by taking a larger window for the moving averages (e.g., 7 days).

In [227]:
a = random.sample(range(100, 1000000), 50000)

In [228]:
%%timeit
moving_average(a, 7)

6.31 s ± 83 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [229]:
%%timeit
a = np.array(a)
moving_average_np(a, 7)

UnboundLocalError: local variable 'a' referenced before assignment

____________________
# `Exercises`
Complete the following short exercises to yet again test your understanding of simple Numpy functions and objects.

Feel free to use the official [documentation](http://docs.scipy.org/doc/) should you need it, and don't worry if you need to google some of the solutions (after all, that's a big part of any programmer's work).

#### ========== Question 1 ==========
Print your numpy version and configuration.

In [ ]:
# Your code goes here
print(np.__version__)
np.show_config()

#### ========== Question 2 ==========
Create a zero vector of size 5.

In [100]:
# Your code goes here
np.zeros(5)

array([0., 0., 0., 0., 0.])

#### ========== Question 3 ==========
Create a zero vector of size 5 of type integer. Set the third element to 1.

In [101]:
# Your code goes here
a = np.zeros(5, dtype=int)
a[2] = 1
a

array([0, 0, 1, 0, 0])

#### ========== Question 4 ==========
Create a vector ranging from 0 to 9. 

In [102]:
# Your code goes here
np.arange(10)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

#### ========== Question 5 ==========
Create a vector ranging from 10 to 29.

In [103]:
# Your code goes here
np.arange(10, 30)

array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
       27, 28, 29])

#### ========== Question 6 ==========
Create a vector ranging from 0 to 9 and reverse it.

In [104]:
# Your code goes here
np.arange(0, 10)[::-1]

array([9, 8, 7, 6, 5, 4, 3, 2, 1, 0])

#### ========== Question 7 ==========
Create a 5 x 3 zero matrix.

In [105]:
# Your code goes here
np.zeros((5, 3))

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

#### ========== Question 8 ==========
Create this matrix...without copy pasting it ;)
```
array([[0, 3, 6],
       [1, 4, 7],
       [2, 5, 8]])
```

In [106]:
# Your code goes here
a = np.arange(9).reshape(3,3)
a.T

array([[0, 3, 6],
       [1, 4, 7],
       [2, 5, 8]])

#### ========== Question 9 ==========
Create a 3 X 3 identity matrix.

In [107]:
# Your code goes here
np.eye(3)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

#### ========== Question 10 ==========
Create a 2 X 2 X 2 array with random values (drawn from a normal distribution).

In [108]:
# Your code goes here
np.random.randn(2, 2, 2)

array([[[ 0.54135978, -0.96034829],
        [ 1.40216585, -0.51139627]],

       [[ 2.29801045, -0.23344737],
        [-1.57633064, -1.56231475]]])

#### ========== Question 11a ==========
Create a 5 x 4 array with random values and find the minimum and maximum values.

In [109]:
# Your code goes here
a = np.random.randn(5, 4)
print(a)
print("Minimum: ", np.min(a))
print("Maximum: ", np.max(a))

[[ 0.41332998  0.17164767  0.7439867  -0.25007344]
 [-0.68904552 -0.81781829  1.6323549  -0.60266574]
 [-0.10519097  0.63381727  0.43157111  0.15112892]
 [ 0.52944019  0.99234884 -0.53288773  0.67670163]
 [ 0.90976072  0.21435408  0.03001567 -0.43312689]]
Minimum:  -0.8178182948037126
Maximum:  1.6323549022582324


#### ========== Question 11b ==========
Return the *index* (i.e. the location within the matrix) of the max or min values

In [110]:
# Your code goes here
idx = a.argmax()    # or...
idx = np.argmax(a)  # ...are acceptable...but a[idx] would fail

In [111]:
np.where(a == a.max())  # is also fine

(array([1]), array([2]))

#### ========== Question 12 ==========
Find the mean value of the array in 11.

In [112]:
# Your code goes here
np.mean(a)

0.20498245317661615

#### ========== Question 13 ==========
Find the row means of the array in 11.

In [113]:
# Your code goes here
np.mean(a, axis=1)

array([ 0.26972273, -0.11929366,  0.27783158,  0.41640073,  0.18025089])

#### ========== Question 14 ==========
Find the column means of the array in 11.

In [114]:
# Your code goes here
np.mean(a, axis=0)

array([ 0.21165888,  0.23886991,  0.46100813, -0.09160711])

#### ========== Question 15 ==========
Create a list with elements 2.2, 3.5, 0, 4, 0. and convert into numpy array. Find the indices of non-zero elements.

In [115]:
# Your code goes here
a = [2.2, 3.5, 0, 4, 0.]
a = np.asarray(a)  # or np.array(a)
np.nonzero(a)

(array([0, 1, 3]),)

#### ========== Question 16 ==========
Crate two normally distributed random matrices of shape (5, 4) and (4, 2). Print their matrix product.

In [116]:
# Your code goes here
a = np.random.randn(5, 4)
b = np.random.randn(4, 2)
np.dot(a,b)

array([[ 0.69909726,  0.7749913 ],
       [-3.10822756, -1.84611092],
       [ 0.97890069,  0.82635873],
       [ 2.12451019,  1.77801163],
       [ 0.12598412, -1.34357738]])

#### ========== Question 17 ==========
Crate a random matrix of shape (5, 3) and a random vector of size 3. Use broadcasting to add the two arrays.

In [117]:
# Your code goes here
a = np.random.randn(5, 3)
b = np.random.randn(3)
a + b

array([[-3.45406079,  0.04060532,  0.10972098],
       [-3.32253751,  1.35125513,  0.0786007 ],
       [-1.95996924,  1.38198164,  0.05889254],
       [-2.31669378,  1.80981971,  0.08496361],
       [-3.61449738, -1.05922466, -0.7984033 ]])

_________
# Endnote & Additional References
_____________
This brief introduction has touched upon most of the important things that you ought to know about NumPy -  a fundamental tool for anyone interested in doing scientific computing in Python. 

However, it offers many more additional operations as optimized computations on multidimensional arrays, so should you want more info or practice, check out the complete [NumPy reference](https://docs.scipy.org/doc/numpy-1.13.0/reference/) as well as the following links:

- [The SciPy NumPy tutorial](http://www.scipy-lectures.org/intro/numpy/index.html)
- [100 Exercises with solutions in Numpy](http://www.labri.fr/perso/nrougier/teaching/numpy.100/)

For a visual introduction to Linear Algebra:
- [Essence of Linear Algebra](https://www.youtube.com/watch?v=kjBOesZCoqc)